# American Options

This model will price and find the optimal stopping times of an **American Put Option** over a $5$ period binomial model with some abritarily decided constants.

## Defining Variables and Formulas

We will define a $5$ period binomial model. The final time period will be called $N$ and we will define $N = 5$.

In [1]:
N = 5

We will define the initial stock price as $S_0 = 4$.

In [2]:
S_0 = 4

The changes in stock price for a binomial model are determined by an imaginary coin flip which results in $H$ or $T$. If the result at time $n$ which we will call $\omega_n$ is $H$ the stock price at time $n-1$ will be multiplied by the up factor $u$ and if $\omega_n = T$ it will be muliplied by the down factor $d$. So, if $\omega_n = H$ then $S_n = uS_{n-1}$ and if $\omega_n = T$ then $S_n = dS_{n-1}$. We will define $u = 2$ and $d = \frac{1}{2}$.

In [3]:
u = 2
d = 1/2

Interest which is assumed to be equal for borrowing and holding money is defined as $r = \frac{1}{4}$.

In [4]:
r = 1/4

For calculating the price of the American Put Option we will use risk-neutral probabilities. The risk-neutral probability of the result of the coin flip at time $n$ to be $\omega_n = H$ is $\tilde{p}$ and the risk-neutral probability of $\omega_n = T$ is $\tilde{q}$. We define $\tilde{p} = \frac{1 + r - d}{u - d}$ and $\tilde{q} = \frac{u - 1 - r}{u - d}$.

In [5]:
def risk_neutral_p(u, d, r):
    return (1 + r - d) / (u - d)

def risk_neutral_q(u, d, r):
    return (u - 1 - r) / (u - d)

rn_p = risk_neutral_p(u, d, r)
rn_q = risk_neutral_p(u, d, r)

We also need to define a strike price for the option which we will define as $K = 5$.

In [6]:
K = 5

## Pricing and Optimal Stopping Times

### Explanation

$$ \textrm{First, we will define } n \textrm{ an arbitrary time in } n = 1, 2, \:.\:.\:.\:, N \textrm{ with } N = 5$$
$$ \textrm{We will define the stock price at time } n \textrm{ as } S_n \textrm{ and strike price as } K$$
$$ \textrm{The value of the option if the option is exercised at time n is defined as } G_n = K - S_n $$
$$ \textrm{The option value at time } n \textrm{ is defined as } V_n = \mathrm{max}\{G_n, \; \tilde{\mathbb{E}}[V_{n+1}]\}$$
$$ \textrm{To define optimal stopping times we use the following formula: } \tau^* = \mathrm{min}\{n; \; V_n = G_n\}$$
$$ \textrm{Thus, the problem can be redefined as } \tau^* = \mathrm{min}\{n; \; G_n \geq \tilde{\mathbb{E}}[V_{n+1}]\}$$
$$ \textrm{We will have to go down every branch of the binomial tree until this condition is met.}$$
$$ \textrm{If this condition is not met we will define } \tau_n(\omega_1, \omega_2, \:.\:.\:.\:, \omega_N) = \infty \textrm{ and the option will never be exercised.}$$
$$ \textrm{Stop times also cannot use future information so if } \tau_n(\omega_1, \omega_2, \:.\:.\:.\:, \omega_{n-1}, H) = 2 \textrm{ then } \tau_n(\omega_1, \omega_2, \:.\:.\:.\:, \omega_{n-1}, T) = 2 \textrm{ must also be true.}$$

### Code

First, I will make a function to get all $H$ and $T$ combinations of length $N$. I will reverse the list of combinations as we will need to work backwards.

In [7]:
from itertools import permutations

def get_combinations(n):
    combinations = []
    for i in range(1, n+1):
        for j in range(i + 1):
            combo = 'H' * j + 'T' * (i - j)
            combinations += list(set([''.join(p) for p in permutations(combo)]))
    return combinations

all_combinations = list(reversed(get_combinations(N)))

Next, I will create a function `G` which will represent the equation $K - S_n$.

In [8]:
def G(stock_price, strike_price):
    return strike_price - stock_price

After that, I will create a function `expected_price` that will get the expected price of an asset in the next time period and then I will create a function `optimal_time_condition` which will represent the condition $\mathrm{max}\{G_n, \; \tilde{\mathbb{E}}[V_{n+1}]\} = \mathrm{max}\{G_n, \; \frac{\tilde{p}V_{n+1}(uS_n) + \tilde{q}V_{n+1}(dS_n)}{1 + r}\}$ which is needed for finding the optimal stopping time.

In [9]:
def expected_price(r, rn_p, rn_q, H_price, T_price):
    return (rn_p * H_price + rn_q * T_price) / (1 + r)

def optimal_time_condition(r, rn_p, rn_q, H_option_price, T_option_price, G):
    return max(G, expected_price(r, rn_p, rn_q, H_option_price, T_option_price))

Next, I will create a function `price_option` that will price all the possible option outcomes and get a price at time $0$. I will define $V_5 = \mathrm{max}\{G_n, 0\}$ for the price at expiration. It will also return a dictionary of intrinsic values so that I can later calculate the optimal stopping times.

In [10]:
def price_option(N, S_0, u, d, rn_p, rn_q, strike_price, all_combinations):
    price_dict = {}
    intrinsic_dict = {}
    for combo in all_combinations:
        new_price = S_0 * u**combo.count('H') * d**combo.count('T')
        if len(combo) == N:
            price_dict[combo] = max(G(new_price, strike_price), 0)
            intrinsic_dict[combo] = G(new_price, strike_price)
            continue
        price_dict[combo] = optimal_time_condition(r, rn_p, rn_q, price_dict[combo + 'H'], price_dict[combo + 'T'], G(new_price, strike_price))
        intrinsic_dict[combo] = G(new_price, strike_price)
    price_dict['0'] = optimal_time_condition(r, rn_p, rn_q, price_dict['H'], price_dict['T'], G(S_0, strike_price))
    intrinsic_dict['0'] = G(S_0, strike_price)
    return price_dict, intrinsic_dict

all_prices, all_intrinsic = price_option(N, S_0, u, d, rn_p, rn_q, K, all_combinations)

Now, we have a value for $V_0$, a time zero price for our American Put Option.

In [11]:
all_prices['0']

1.45344

Next, I will create a function `get_stopping_times` that will output a dictionary containing the stopping times for each $H$ and $T$ combination. I will represent $\infty$ as `None` in the dictionary. The function will simply return $0$ if it is optimal to exercise the option at time $0$ and not hold it. Only stopping times for combinations of length $N$ will be given.

In [18]:
def get_stopping_times(N, prices, intrinsic_values):
    stopping_points = []
    stopping_times = {}
    for combo in reversed(prices.keys()):
        if prices[combo] == intrinsic_values[combo]:
            if combo == '0':
                return 0
            stopping_points.append(combo)
        if len(combo) == N:
            stopping_times[combo] = None
            part_combo = ""
            for char in combo:
                part_combo += char
                if part_combo in stopping_points:
                    stopping_times[combo] = len(part_combo)
                    break
    print(stopping_points)
    return stopping_times
                
            
                
            

get_stopping_times(N, all_prices, all_intrinsic)

['T', 'TT', 'TTT', 'THT', 'HTT', 'TTH', 'TTTT', 'THTT', 'TTHT', 'HTTT', 'TTTH', 'TTTTT', 'TTTTH', 'THTTT', 'HTTTT', 'TTTHT', 'TTHTT', 'HTHTT', 'HHTTT', 'THTTH', 'HTTTH', 'THHTT', 'THTHT', 'TTHHT', 'TTHTH', 'HTTHT', 'TTTHH']


{'TTTTT': 1,
 'TTTTH': 1,
 'THTTT': 1,
 'HTTTT': 3,
 'TTTHT': 1,
 'TTHTT': 1,
 'HTHTT': 5,
 'HHTTT': 5,
 'THTTH': 1,
 'HTTTH': 3,
 'THHTT': 1,
 'THTHT': 1,
 'TTHHT': 1,
 'TTHTH': 1,
 'HTTHT': 3,
 'TTTHH': 1,
 'HHHTT': None,
 'HHTTH': None,
 'HTHTH': None,
 'HTHHT': None,
 'THHTH': 1,
 'THHHT': 1,
 'HTTHH': 3,
 'THTHH': 1,
 'HHTHT': None,
 'TTHHH': 1,
 'HHTHH': None,
 'HHHHT': None,
 'HTHHH': None,
 'HHHTH': None,
 'THHHH': 1,
 'HHHHH': None}

And below we can see the optimal stopping time for each $H$ and $T$ combination.